In [1]:
import torch
torch.__version__

'1.12.1+cu102'

In [2]:
!pip install torch-scatter -f https://data.pyg.org/whl/torch-1.12.1+cu102.html
!pip install torch-sparse -f https://data.pyg.org/whl/torch-1.12.1+cu102.html
!pip install torch-geometric

Defaulting to user installation because normal site-packages is not writeable
Looking in links: https://data.pyg.org/whl/torch-1.12.1+cu102.html
Defaulting to user installation because normal site-packages is not writeable
Looking in links: https://data.pyg.org/whl/torch-1.12.1+cu102.html
Defaulting to user installation because normal site-packages is not writeable


In [84]:
import os.path as osp

import numpy as np
import pandas as pd
import csv
import torch
from torch_geometric.data import Dataset, download_url, Data


class NodeDataset(Dataset):
    def __init__(self, count_rows = None, root = "../dataset/", transform=None, pre_transform=None, pre_filter=None):
        self.sequence_predict = []
        self.files_data = []
        self.count_rows = count_rows
        
        super().__init__(root, transform, pre_transform, pre_filter)

    @property
    def raw_file_names(self):
        return ['FINAL_ALL_SEQUENCES_TRAINTEST.tsv', 'FINAL_FEATURES_FRIENDS.tsv', 'FINAL_FEATURES_TRAINTEST.tsv', 'FINAL_TARGETS_DATES_TRAINTEST.tsv']

    @property
    def processed_file_names(self):
        return self.files_data
    
    def download(self):
        return ['FINAL_ALL_SEQUENCES_TRAINTEST.tsv', 'FINAL_FEATURES_FRIENDS.tsv', 'FINAL_FEATURES_TRAINTEST.tsv', 'FINAL_TARGETS_DATES_TRAINTEST.tsv']
    
    def process_sequence(self, sequence, latent_space = 50):
        processed_sequence = torch.from_numpy(np.zeros((latent_space)))
        
        self.sequence_predict.append(processed_sequence)
        
    
    def process(self):
        # последовательности пользовательских состояний для пользователей из основной выборки
        ALL_SEQUENCES_TRAINTEST = self.root + '/FINAL_ALL_SEQUENCES_TRAINTEST.tsv'

        # связь между пользователем из основной выборки и всеми его друзьями (граф) + те же самые два домена признаков для друзей
        FEATURES_FRIENDS = self.root + '/FINAL_FEATURES_FRIENDS.tsv'

        # 2 домена вещественных признаков
        FEATURES_TRAINTEST = self.root + '/FINAL_FEATURES_TRAINTEST.tsv'

        # метки классов, ретродаты и идентификаторы тестовых объектов
        TARGETS_DATES_TRAINTEST = self.root + '/FINAL_TARGETS_DATES_TRAINTEST.tsv'
        
        ####DATAFRAMES####
        
        client_sequences = pd.read_csv(ALL_SEQUENCES_TRAINTEST,sep='\t')
        
        targets = pd.read_csv(TARGETS_DATES_TRAINTEST,sep='\t')
        
        features = pd.read_csv(FEATURES_TRAINTEST,sep='\t')
        
        ####Process graph####
                
        client_set = set()    
        
        
        dataframe_friends = pd.DataFrame()
        chunksize = 100
        rows = 0
        idx = 0
        for friends_chunk in pd.read_csv(FEATURES_FRIENDS, sep='\t', chunksize=chunksize, nrows=self.count_rows):
            for row in friends_chunk.iloc():
                rows += 1;
                if rows % 10000 == 0:
                    if self.count_rows == None: print(rows)
                    else:  print(float(rows) / float(self.count_rows) * 100, "%")
                if row['CLIENT_ID'] in client_set:
                    dataframe_friends = pd.concat([dataframe_friends, row], axis=1)
                else:
                    if len(client_set) > 0:
                        client_feature = torch.tensor([features[features['CLIENT_ID'] == row['CLIENT_ID']].values[0][1:]])
                        friend_feature = torch.tensor(dataframe_friends.values)[1:-1]
                        
                        if len(friend_feature.shape) == 1: friend_feature = friend_feature.reshape(1014, -1)
                        
                        friend_feature = friend_feature.transpose(0, 1)
                        
                        X = torch.cat((client_feature, friend_feature))
                        
                        edges = torch.from_numpy(np.concatenate((np.zeros((len(friend_feature))), np.arange(1,len(friend_feature) + 1))).reshape((2, -1))).int()
                        edge_attrs = torch.tensor(dataframe_friends.values)[-1] % 100
                                                
                        sequence = client_sequences[client_sequences['CLIENT_ID'] == row['CLIENT_ID']]['SEQUENCE'].values
                        self.process_sequence(sequence)
                        
                        predict = targets[targets['CLIENT_ID'] == row['CLIENT_ID']]['TARGET'].values
                        
                        if predict[0] == 'test': predict[0] = -1
                        else: predict[0] = float(predict[0])
                        
                        predict = torch.from_numpy(np.array(predict).astype(float))
                        
                        data = Data(
                            x = X,
                            edge_index = edges,
                            edge_attr = edge_attrs,
                            y = predict
                        )
                        
                        torch.save(data, osp.join(self.processed_dir, f'data_{idx}.pt'))
                        self.files_data.append(osp.join(self.processed_dir, f'data_{idx}.pt'))
                        idx += 1
                        
                    client_set.add(row['CLIENT_ID'])
                    dataframe_friends = row
                    
            

    def len(self):
        return len(self.files_data)

    def get(self, idx):
        data = torch.load(osp.join(self.processed_dir, f'data_{idx}.pt'))
        return data
    
ds = NodeDataset(20000)

Processing...


50.0 %
100.0 %


Done!


In [82]:
len(ds)

1294

In [79]:
from torch_geometric.loader import DataLoader

loader = DataLoader(ds, batch_size=4, shuffle=True)

In [80]:
next(iter(loader))

DataBatch(x=[265, 1014], edge_index=[2, 261], edge_attr=[261], y=[4], batch=[265], ptr=[5])

In [97]:
import os.path as osp

import numpy as np
import pandas as pd
import csv
import torch
from torch_geometric.data import Dataset, download_url, Data


class TestDataset(Dataset):
    def __init__(self, count_rows = None, root = "../dataset/", transform=None, pre_transform=None, pre_filter=None):
        self.sequence_predict = []
        self.files_data = []
        self.count_rows = count_rows
        
        super().__init__(root, transform, pre_transform, pre_filter)

    @property
    def raw_file_names(self):
        return ['FINAL_ALL_SEQUENCES_TRAINTEST.tsv', 'FINAL_FEATURES_FRIENDS.tsv', 'FINAL_FEATURES_TRAINTEST.tsv', 'FINAL_TARGETS_DATES_TRAINTEST.tsv']

    @property
    def processed_file_names(self):
        return self.files_data
    
    def download(self):
        return ['FINAL_ALL_SEQUENCES_TRAINTEST.tsv', 'FINAL_FEATURES_FRIENDS.tsv', 'FINAL_FEATURES_TRAINTEST.tsv', 'FINAL_TARGETS_DATES_TRAINTEST.tsv']
    
    def process_sequence(self, sequence, latent_space = 50):
        processed_sequence = torch.from_numpy(np.zeros((latent_space)))
        
        self.sequence_predict.append(processed_sequence)
        
    
    def process(self):
        # последовательности пользовательских состояний для пользователей из основной выборки
        ALL_SEQUENCES_TRAINTEST = self.root + '/FINAL_ALL_SEQUENCES_TRAINTEST.tsv'

        # связь между пользователем из основной выборки и всеми его друзьями (граф) + те же самые два домена признаков для друзей
        FEATURES_FRIENDS = self.root + '/FINAL_FEATURES_FRIENDS.tsv'

        # 2 домена вещественных признаков
        FEATURES_TRAINTEST = self.root + '/FINAL_FEATURES_TRAINTEST.tsv'

        # метки классов, ретродаты и идентификаторы тестовых объектов
        TARGETS_DATES_TRAINTEST = self.root + '/FINAL_TARGETS_DATES_TRAINTEST.tsv'
        
        ####DATAFRAMES####
        
        client_sequences = pd.read_csv(ALL_SEQUENCES_TRAINTEST,sep='\t')
        
        targets = pd.read_csv(TARGETS_DATES_TRAINTEST,sep='\t')
        
        features = pd.read_csv(FEATURES_TRAINTEST,sep='\t')
        
        ####Process graph####
                
        client_set = list(set(targets[targets['TARGET'] == "test"]['CLIENT_ID'].values))
        #print(client_set)
        visited = []
        in_progress = []
        
        dataframe_friends = pd.DataFrame()
        chunksize = 1000
        rows = 0
        idx = 0
        
        self.dict_id_index = {}
        
        for friends_chunk in pd.read_csv(FEATURES_FRIENDS, sep='\t', chunksize=chunksize, nrows=self.count_rows):
            print(float(len(visited)) / len(client_set) * 100, "%")
            for row in friends_chunk.iloc():
                if row['CLIENT_ID'] in client_set:
                    if row['CLIENT_ID'] not in visited:
                        if row['CLIENT_ID'] in in_progress:
                            dataframe_friends = pd.concat([dataframe_friends, row], axis=1)
                        else:
                            if(len(in_progress) > 0):
                                client_feature = torch.tensor([features[features['CLIENT_ID'] == in_progress[0]].values[0][1:]])
                                friend_feature = torch.tensor(dataframe_friends.values)[1:-1]

                                if len(friend_feature.shape) == 1: friend_feature = friend_feature.reshape(1014, -1)

                                friend_feature = friend_feature.transpose(0, 1)

                                X = torch.cat((client_feature, friend_feature))

                                edges = torch.from_numpy(np.concatenate((np.zeros((len(friend_feature))), np.arange(1,len(friend_feature) + 1))).reshape((2, -1))).int()
                                edge_attrs = torch.tensor(dataframe_friends.values)[-1] % 100

                                sequence = client_sequences[client_sequences['CLIENT_ID'] == in_progress[0]]['SEQUENCE'].values
                                self.process_sequence(sequence)

                                data = Data(
                                    x = X,
                                    edge_index = edges,
                                    edge_attr = edge_attrs,
                                )


                                torch.save(data, osp.join(self.processed_dir, f'data_test_{idx}.pt'))
                                self.files_data.append(osp.join(self.processed_dir, f'data_test_{idx}.pt'))
                                self.dict_id_index[in_progress[0]] = idx
                                print(in_progress[0])
                                print(in_progress[0] in client_set)
                                idx += 1
                                visited.append(in_progress[0])
                            dataframe_friends = row
                            in_progress = []
                            in_progress.append(row['CLIENT_ID'])
    def len(self):
        return len(self.files_data)

    def get(self, idx):
        data = torch.load(osp.join(self.processed_dir, f'data_test_{idx}.pt'))
        return data
    
test_ds = TestDataset(20000)

Processing...


0.0 %
999555.0
True
0.0031389289974260784 %
998390.0
True
99413.0
True
0.009416786992278235 %
0.009416786992278235 %
993533.0
True
988262.0
True
988195.0
True
985939.0
True
0.02197250298198255 %
985543.0
True
985090.0
True
982512.0
True
0.031389289974260784 %
982046.0
True
981611.0
True
980977.0
True
97974.0
True
0.0439450059639651 %
979393.0
True
974267.0
True
0.050222863958817254 %
972275.0
True
968052.0
True
967424.0
True
0.05963965095109548 %
964825.0
True
962881.0
True
960522.0
True
960503.0
True
0.0721953669407998 %
960440.0
True
959656.0
True
0.07847322493565195 %
957644.0
True
952574.0
True
0.08475108293050411 %
951618.0
True
949740.0
True
947786.0
True
947076.0
True
0.09730679892020842 %
945573.0
True
945489.0
True
944949.0
True
0.10672358591248667 %
943087.0
True
940606.0
True
940005.0
True
939914.0
True
0.11927930190219097 %
939386.0
True
93520.0
True
934635.0
True
933894.0
True
0.13183501789189528 %
932918.0
True
0.13497394688932138 %
0.13497394688932138 %
923618.0
True
915

Done!


In [98]:
len(test_ds)

53

In [99]:
test_ds.dict_id_index

{999555.0: 0,
 998390.0: 1,
 99413.0: 2,
 993533.0: 3,
 988262.0: 4,
 988195.0: 5,
 985939.0: 6,
 985543.0: 7,
 985090.0: 8,
 982512.0: 9,
 982046.0: 10,
 981611.0: 11,
 980977.0: 12,
 97974.0: 13,
 979393.0: 14,
 974267.0: 15,
 972275.0: 16,
 968052.0: 17,
 967424.0: 18,
 964825.0: 19,
 962881.0: 20,
 960522.0: 21,
 960503.0: 22,
 960440.0: 23,
 959656.0: 24,
 957644.0: 25,
 952574.0: 26,
 951618.0: 27,
 949740.0: 28,
 947786.0: 29,
 947076.0: 30,
 945573.0: 31,
 945489.0: 32,
 944949.0: 33,
 943087.0: 34,
 940606.0: 35,
 940005.0: 36,
 939914.0: 37,
 939386.0: 38,
 93520.0: 39,
 934635.0: 40,
 933894.0: 41,
 932918.0: 42,
 923618.0: 43,
 915834.0: 44,
 914539.0: 45,
 913323.0: 46,
 913153.0: 47,
 912638.0: 48,
 912075.0: 49,
 911916.0: 50,
 908598.0: 51,
 908597.0: 52}

In [ ]:
31858